<a href="https://colab.research.google.com/github/maulanafaiz122/PPW-A/blob/main/Maulana_Faizal_F_21_081_Tugas_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama : Maulana Faizal Fardani

NIM : 210411100081

Kelas : PPW A

In [15]:
import string
import pandas as pd
import numpy as np

In [16]:
import requests
from bs4 import BeautifulSoup
import csv

# URL dari artikel sepak bola di IDN Times
url = 'https://www.idntimes.com/sport/arena/sandy-firdaus/perenang-indonesia-jajal-danau-worthersee-di-austria-tempuh-34-km'

# Membuat permintaan ke URL
response = requests.get(url)

# Inisialisasi data yang akan disimpan
# Mengambil ulang dan membersihkan data yang diminta
artikel_mees = []

if response.status_code == 200:
    # Parsing halaman web
    soup = BeautifulSoup(response.text, 'html.parser')

    # Mengambil judul artikel
    title = soup.find('h1').get_text().strip()

    # Mengambil tanggal publikasi artikel dan membersihkan spasi berlebih
    date = soup.find('time').get_text().strip()

    # Mengambil isi artikel dengan membersihkan konten tambahan
    content = soup.find_all('p')
    article_text = '\n'.join([p.get_text().strip() for p in content if p.get_text().strip()])

    # Simpan data dalam bentuk dictionary yang telah dibersihkan
    artikel_mees.append({
        'Title': title,
        'Date': date,
        'Content': article_text
    })

# Membuat dataframe dari data yang telah dibersihkan
data_mees = pd.DataFrame(artikel_mees)

# Menampilkan dataframe yang bersih
data_mees

Title               Date  \
0  Perenang Indonesia Jajal Danau Worthersee di A...  10 Oct 24 | 08:35   

                                             Content  
0  Regional\nKategori\nEvent\nDOWNLOAD IDN APP SE...

In [17]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
# Proses case folding
def casefolding(Content):
    if isinstance(Content, list):
        # Jika Content adalah list, terapkan case folding pada setiap elemen
        return [str(item).lower() for item in Content]
    else:
        # Jika Content adalah string, langsung terapkan case folding
        return str(Content).lower()

# Terapkan fungsi casefolding pada kolom 'Content'
data_mees['Content'] = data_mees['Content'].apply(casefolding)

data_mees.head()

Title               Date  \
0  Perenang Indonesia Jajal Danau Worthersee di A...  10 Oct 24 | 08:35   

                                             Content  
0  regional\nkategori\nevent\ndownload idn app se...

In [19]:
csv_filename = "artikel_mees_hilgers.csv"
data_mees.to_csv(csv_filename, index=False, encoding='utf-8')

print(f"Data berita telah disimpan ke {csv_filename}")

Data berita telah disimpan ke artikel_mees_hilgers.csv


In [20]:
data_mees = pd.read_csv("artikel_mees_hilgers.csv", sep=',', encoding='latin1')
data_mees.head()

Title               Date  \
0  Perenang Indonesia Jajal Danau Worthersee di A...  10 Oct 24 | 08:35   

                                             Content  
0  regional\nkategori\nevent\ndownload idn app se...

In [21]:
def cleansing(Content):
    # Menghapus whitespace di awal dan akhir teks
    Content = Content.strip()

    # Menghapus tanda baca dan karakter khusus
    Content = re.sub(f"[{string.punctuation}]", '', Content)

    # Menghapus angka
    Content = re.sub(r'\d+', '', Content)

    # Menghapus huruf tunggal (opsional, tergantung kebutuhan)
    Content = re.sub(r"\b[a-zA-Z]\b", "", Content)

    # Menghapus karakter-karakter non-standar seperti â
    Content = re.sub(r'[^\x00-\x7F]+', '', Content)

    # Menghapus spasi ganda
    Content = re.sub(r'\s+', ' ', Content)

    return Content

# Terapkan fungsi cleansing pada kolom 'Content'
data_mees['Content'] = data_mees['Content'].apply(cleansing)

# Menampilkan 5 baris pertama
data_mees.head(5)

Title               Date  \
0  Perenang Indonesia Jajal Danau Worthersee di A...  10 Oct 24 | 08:35   

                                             Content  
0  regional kategori event download idn app sekar...

In [22]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [23]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd  # Ensure you import pandas for DataFrame usage

# Download required resources
nltk.download('punkt')
nltk.download('punkt_tab')

# Assume article_text is defined; replace with your actual text
# article_text = "Your article text goes here."

# Ekstraksi kalimat dari konten artikel
sentences = sent_tokenize(article_text)

# Tokenisasi tiap kalimat menjadi term
terms_per_sentence = [word_tokenize(sentence) for sentence in sentences]

# Membuat objek yang menyimpan hasil term dari setiap kalimat
extracted_terms = [{'Sentence': sentence, 'Terms': terms} for sentence, terms in zip(sentences, terms_per_sentence)]

# Menampilkan hasilnya sebagai dataframe
df_terms = pd.DataFrame(extracted_terms)

# Menampilkan dataframe
print(df_terms)


                                             Sentence  \
0   Regional\nKategori\nEvent\nDOWNLOAD IDN APP SE...   
1   Jakarta, IDN Times - Perenang Indonesia, Benha...   
2                  Dia berenang selama hampir 12 jam.   
3   Docben, sapaan akrab Benhart, melakukan ini pa...   
4   Hal ini merupakan upayanya dalam memperkenalka...   
5   Docben mengungkapkan, tidak mudah bisa terpili...   
6   "Peserta harus meyakinkan panitia dengan menya...   
7                                   Baca Juga: Rekor!   
8   Ratusan Perenang Bentangkan Merah Putih 8 Km d...   
9   Latihannya pun bervariasi, bahkan sampai menem...   
10  "Variasi program latihan renang harian berjara...   
11  Setiba di Austria seminggu sebelum lomba, Docb...   
12  Gangguan pun sempat melanda, termasuk hujan da...   
13  "Gangguan hujan dan badai ringan sebelum hari ...   
14  Keberhasilan di Austria ini menambah pencapaia...   
15  September 2023 silam, dia jadi orang Indonesia...   
16  Docben menempuh renang di l

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [24]:
csv_filename = "artikel_mees_hilgers_ekstract_term.csv"
df_terms.to_csv(csv_filename, index=False, encoding='utf-8')

print(f"Data berita telah disimpan ke {csv_filename}")

Data berita telah disimpan ke artikel_mees_hilgers_ekstract_term.csv


In [25]:
pip install Sastrawi

In [26]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize, word_tokenize

# Assuming we have the article_text
sentences = sent_tokenize(article_text)
terms_per_sentence = [word_tokenize(sentence) for sentence in sentences]
extracted_terms = [{'Sentence': sentence, 'Terms': terms} for sentence, terms in zip(sentences, terms_per_sentence)]
df_terms = pd.DataFrame(extracted_terms)

# Menggabungkan terms menjadi string untuk setiap kalimat
df_terms['Terms_String'] = df_terms['Terms'].apply(' '.join)

# Membuat TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit dan transform data
tfidf_matrix = tfidf_vectorizer.fit_transform(df_terms['Terms_String'])

# Mendapatkan nama-nama feature (terms)
feature_names = tfidf_vectorizer.get_feature_names_out()

# Membuat DataFrame untuk nilai TF-IDF
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

# Menambahkan kolom Sentence
tfidf_df['Sentence'] = df_terms['Sentence']

# Melelehkan (melting) DataFrame untuk format yang lebih mudah dibaca
melted_tfidf = tfidf_df.melt(id_vars=['Sentence'], var_name='Term', value_name='TF-IDF')

# Menghapus baris dengan nilai TF-IDF 0
melted_tfidf = melted_tfidf[melted_tfidf['TF-IDF'] != 0]

# Mengurutkan berdasarkan nilai TF-IDF tertinggi
melted_tfidf = melted_tfidf.sort_values('TF-IDF', ascending=False)

# Menampilkan hasil
print(melted_tfidf)

                                               Sentence        Term    TF-IDF
2842                                  Baca Juga: Rekor!       rekor  0.626081
1582                                  Baca Juga: Rekor!        juga  0.551372
427                                   Baca Juga: Rekor!        baca  0.551372
2555  Keberhasilan di Austria ini menambah pencapaia...  pencapaian  0.487154
2072  Keberhasilan di Austria ini menambah pencapaia...    menambah  0.487154
...                                                 ...         ...       ...
1058  Ratusan Perenang Bentangkan Merah Putih 8 Km d...      docben  0.107077
1060  "Variasi program latihan renang harian berjara...      docben  0.105091
1063  "Gangguan hujan dan badai ringan sebelum hari ...      docben  0.099276
1055  Docben mengungkapkan, tidak mudah bisa terpili...      docben  0.098899
1056  "Peserta harus meyakinkan panitia dengan menya...      docben  0.090688

[312 rows x 3 columns]


In [27]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cosine

# Assuming we have the melted_tfidf DataFrame from the previous code
# If not, uncomment and run the previous code to get melted_tfidf

# Step 1: Pivot the melted DataFrame back to wide format
tfidf_wide = melted_tfidf.pivot(index='Sentence', columns='Term', values='TF-IDF').fillna(0)

# Step 2: Calculate cosine similarity between sentences
num_sentences = len(tfidf_wide)
adjacency_matrix = np.zeros((num_sentences, num_sentences))

for i in range(num_sentences):
    for j in range(i, num_sentences):  # We only need to calculate upper triangle
        if i == j:
            adjacency_matrix[i][j] = 1.0  # Sentence is fully similar to itself
        else:
            similarity = 1 - cosine(tfidf_wide.iloc[i], tfidf_wide.iloc[j])
            adjacency_matrix[i][j] = similarity
            adjacency_matrix[j][i] = similarity  # Matrix is symmetric

# Create a DataFrame for better visualization
adjacency_df = pd.DataFrame(adjacency_matrix,
                            index=tfidf_wide.index,
                            columns=tfidf_wide.index)

# Display the adjacency matrix
print("Adjacency Matrix:")
print(adjacency_df)

# Optional: You can set a threshold to consider only strong connections
threshold = 0.5
adjacency_df_thresholded = adjacency_df.where(adjacency_df > threshold, 0)

print("\nAdjacency Matrix (with threshold):")
print(adjacency_df_thresholded)

# Optional: Save to CSV
adjacency_df.to_csv('adjacency_matrix.csv')
adjacency_df_thresholded.to_csv('adjacency_matrix_thresholded.csv')

Adjacency Matrix:
Sentence                                            "Gangguan hujan dan badai ringan sebelum hari kompetisi, menyebabkan jadwal renang yang sebelumnya ditetapkan pada tanggal 6 September akhirnya harus diundur menjadi tanggal 8 September," kata Docben.  \
Sentence                                                                                                                                                                                                                                                       
"Gangguan hujan dan badai ringan sebelum hari k...                                           1.000000                                                                                                                                                          
"Peserta harus meyakinkan panitia dengan menyam...                                           0.084237                                                                                                                 

In [28]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import textwrap

# Assuming we have the tfidf_wide DataFrame and original sentences from previous steps
# If not, you need to run the TF-IDF calculation code first

# Calculate cosine similarity
cosine_sim = cosine_similarity(tfidf_wide)

# Create a DataFrame for the similarity matrix
similarity_df = pd.DataFrame(cosine_sim, index=tfidf_wide.index, columns=tfidf_wide.index)

# Create a graph from the similarity matrix
G = nx.from_pandas_adjacency(similarity_df)

# Remove self-loops
G.remove_edges_from(nx.selfloop_edges(G))

# Function to get the largest connected component
def get_largest_component(G):
    return max(nx.connected_components(G), key=len)

# Ensure the graph is connected
while not nx.is_connected(G):
    largest_component = get_largest_component(G)
    isolated_nodes = set(G.nodes()) - set(largest_component)
    if not isolated_nodes:
        break
    for node in isolated_nodes:
        similarities = [(other_node, similarity_df.loc[node, other_node])
                        for other_node in largest_component]
        most_similar_node = max(similarities, key=lambda x: x[1])[0]
        G.add_edge(node, most_similar_node)

# Function to wrap text
def wrap_text(text, width=20):
    return '\n'.join(textwrap.wrap(text, width=width))

# Prepare labels (wrapped sentences)
labels = {node: wrap_text(node) for node in G.nodes()}

# Now let's visualize the graph
plt.figure(figsize=(20, 16))  # Increased figure size
pos = nx.spring_layout(G, k=0.5, iterations=50)

# Calculate node sizes based on degree centrality
degree_centrality = nx.degree_centrality(G)
node_sizes = [v * 10000 for v in degree_centrality.values()]  # Increased node sizes

# Draw the graph
nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color='lightblue')
nx.draw_networkx_edges(G, pos, alpha=0.3)

# Add labels
nx.draw_networkx_labels(G, pos, labels, font_size=6)  # Reduced font size

plt.title("Connected Sentence Similarity Graph")
plt.axis('off')
plt.tight_layout()

# Save the graph as an image
plt.savefig('connected_sentence_similarity_graph.png', dpi=300, bbox_inches='tight')

# Show the plot
plt.show()

# Print some information about the graph
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")
print(f"Is the graph connected? {nx.is_connected(G)}")

# Calculate centrality measures
closeness_centrality = nx.closeness_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)

# Create a DataFrame with centrality measures
centrality_data = []
for node in G.nodes():
    centrality_data.append({
        'Sentence': node,
        'Degree Centrality': degree_centrality[node],
        'Closeness Centrality': closeness_centrality[node],
        'Betweenness Centrality': betweenness_centrality[node]
    })

centrality_df = pd.DataFrame(centrality_data)

# Sort by Degree Centrality
centrality_df = centrality_df.sort_values('Degree Centrality', ascending=False)

print("\nCentrality Measures:")
print(centrality_df)

# Save centrality measures to CSV
centrality_df.to_csv('centrality_measures_with_sentences.csv', index=False)

# Optionally, print the top 5 sentences by each centrality measure
print("\nTop 5 sentences by Degree Centrality:")
print(centrality_df.nlargest(5, 'Degree Centrality')[['Sentence', 'Degree Centrality']])

print("\nTop 5 sentences by Closeness Centrality:")
print(centrality_df.nlargest(5, 'Closeness Centrality')[['Sentence', 'Closeness Centrality']])

print("\nTop 5 sentences by Betweenness Centrality:")
print(centrality_df.nlargest(5, 'Betweenness Centrality')[['Sentence', 'Betweenness Centrality']])

Number of nodes: 21
Number of edges: 133
Is the graph connected? True

Centrality Measures:
                                             Sentence  Degree Centrality  \
17  Ratusan Perenang Bentangkan Merah Putih 8 Km d...               0.90   
16  Pada Agustus 2024, MURI kemudian kembali mengg...               0.85   
7   Docben menempuh renang di laut tersebut selama...               0.85   
0   "Gangguan hujan dan badai ringan sebelum hari ...               0.80   
19  September 2023 silam, dia jadi orang Indonesia...               0.80   
6   Dia berenang tanpa alat sejauh 23 km di kolam ...               0.80   
8   Docben mengungkapkan, tidak mudah bisa terpili...               0.80   
12  Jakarta, IDN Times - Perenang Indonesia, Benha...               0.80   
11  Hal ini merupakan upayanya dalam memperkenalka...               0.75   
13  Keberhasilan di Austria ini menambah pencapaia...               0.75   
20  Setiba di Austria seminggu sebelum lomba, Docb...               0.75